<a href="https://colab.research.google.com/github/monuminu/affine_hackathon/blob/master/key_value_pair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import argparse
import numpy as np
!pip install opencv-wrapper
import opencv_wrapper as cvw
from skimage.filters import threshold_local
import json

     |████████████████████████████████| 17.3MB 67.7MB/s 
     |████████████████████████████████| 25.4MB 114kB/s 
  Created wheel for opencv-wrapper: filename=opencv_wrapper-0.2.3-py2.py3-none-any.whl size=18581 sha256=3e0cbfc56d16788f3760fd36dab37e66ea7ec2b477c5c1cb8d4f3f631eacbd60
  Stored in directory: /root/.cache/pip/wheels/74/fc/22/03106b3bcb2516e69317562a2aa4ceb94b0d5813a448edd928
Successfully built opencv-wrapper
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.16.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalle

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

!pip install -U git+https://github.com/madmaze/pytesseract.git

import pytesseract
!sudo apt install tesseract-ocr
!pip install colorama

Mounted at /content/gdrive
  Cloning https://github.com/madmaze/pytesseract.git to /tmp/pip-req-build-1y03g5zn
  Running command git clone -q https://github.com/madmaze/pytesseract.git /tmp/pip-req-build-1y03g5zn
  Created wheel for pytesseract: filename=pytesseract-0.3.6-py2.py3-none-any.whl size=13690 sha256=0db66733c15dd93c75602442ef3b11fb1ba979af514eb3b1564e9fd80e242765
  Stored in directory: /tmp/pip-ephem-wheel-cache-y5mnjmpd/wheels/be/2a/a1/a40dbc7e579dffb2be8bbc3243c491de2d132899309f008b1f
Successfully built pytesseract
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 11 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://a

In [3]:

# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required = True, help = "Path to the image to be scanned")
# ap.add_argument("-o", "--output", required = True, help = "Path to the image to be scanned")
# args = vars(ap.parse_args())


def get_acc(directory, path):
    font     = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.5
    fontColor  = (255,0,0)
    lineType = 1
    path = directory+path
    # path = args['image']
    # op_path = args['output']

    op_path = directory
    if op_path[-1]!='/':
    	op_path.append('/')


    #Threshold
    image = cv2.imread(path)

    height,width,channel = image.shape

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    T = threshold_local(gray, 15, offset = 6, method = "gaussian") # generic, mean, median, gaussian
    thresh = (gray > T).astype("uint8") * 255
    thresh = ~thresh

    #cv2.imwrite(op_path+'threshold.png', thresh)

    #Dilation
    kernel =np.ones((1,1), np.uint8)
    ero = cv2.erode(thresh, kernel, iterations= 1)
    img_dilation = cv2.dilate(ero, kernel, iterations=1)
    #cv2.imwrite(op_path+'dilated.png', img_dilation)

    # Remove noise
    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(img_dilation, None, None, None, 8, cv2.CV_32S)
    sizes = stats[1:, -1] #get CC_STAT_AREA component
    final = np.zeros((labels.shape), np.uint8)
    for i in range(0, nlabels - 1):
        if sizes[i] >= 10:   #filter small dotted regions
            final[labels == i + 1] = 255
    #cv2.imwrite(op_path+'final.png', final)
    #Find contours
    kern = np.ones((5,15), np.uint8)
    img_dilation = cv2.dilate(final, kern, iterations = 1)
    #cv2.imwrite(op_path+'contours.png', img_dilation)
    contours, hierarchy = cv2.findContours(img_dilation, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    # Map contours to bounding rectangles, using bounding_rect property
    rects = map(lambda c: cv2.boundingRect(c), contours)
    # Sort rects by top-left x (rect.x == rect.tl.x)
    sorted_rects = sorted(rects, key =lambda r: r[0])
    sorted_rects = sorted(sorted_rects, key =lambda r: r[1])

    tt = image.copy()
    dictionary = {}
    etfo=''
    for i,rect in enumerate(sorted_rects):
        temp_dic = {}
        x,y,w,h = rect
        if(w<20 or h<20):
            continue
        temp_dic['coords'] = [x,y,w,h]
        words = []
        temp = tt[y:y+h, x:x+w]
        #cv2.imwrite('/content/gdrive/My Drive/ResearchPaper/AIESI_complete_pipeline/temp/'+str(i)+'.png',temp)
        temp = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
        hi = pytesseract.image_to_data(temp, config=r'--psm 6')
        hi = hi.split()
        ind = 22
        while(True):
            if (ind>len(hi)):
                break
            if(int(hi[ind])==-1):
                ind+=11
            else:
                #cv2.putText(image,hi[ind]+','+hi[ind+1], (x,y), font, fontScale,fontColor,lineType)
                tem = {}
                tem['confidence'] = hi[ind]
                tem['text'] = hi[ind+1]
                etfo=etfo+hi[ind+1]
                etfo=etfo+" "
                x+=len(hi[ind+1])*20
                ind+=12
                words.append(tem)
        temp_dic['words'] = words
        etfo=etfo+'\n'
        #cvw.rectangle(image, rect, cvw.Color.GREEN, thickness=1)
        dictionary[i] = temp_dic


    cv2.imwrite(op_path+"result.png", image)
    return json.dumps(dictionary),etfo


In [68]:

gson_data, etfo = get_acc('/content/gdrive/My Drive/datascience/','098.jpg')
print(gson_data)


{"9": {"coords": [66, 56, 37, 46], "words": [{"confidence": "76", "text": "3"}]}, "10": {"coords": [128, 61, 20, 29], "words": []}, "11": {"coords": [159, 61, 90, 43], "words": [{"confidence": "14", "text": "O"}, {"confidence": "14", "text": "td"}]}, "12": {"coords": [250, 66, 98, 36], "words": [{"confidence": "33", "text": "OF?"}]}, "15": {"coords": [506, 129, 44, 33], "words": [{"confidence": "40", "text": "il"}, {"confidence": "0", "text": "ft"}]}, "20": {"coords": [443, 220, 43, 32], "words": []}, "22": {"coords": [57, 306, 88, 30], "words": [{"confidence": "91", "text": "WAHIN"}]}, "23": {"coords": [148, 308, 134, 30], "words": [{"confidence": "95", "text": "HARDWARE"}]}, "24": {"coords": [285, 309, 58, 29], "words": [{"confidence": "79", "text": "SIN"}]}, "25": {"coords": [346, 309, 57, 29], "words": [{"confidence": "87", "text": "BHO"}]}, "26": {"coords": [59, 342, 132, 35], "words": [{"confidence": "89", "text": "148481-U"}]}, "27": {"coords": [58, 387, 69, 36], "words": [{"con

In [69]:
import argparse
import torch
import json
from torch import nn

class MyModel0(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=2, bidirectional=True)
        self.linear = nn.Linear(hidden_size * 2, 5)

    def forward(self, inpt):
        embedded = self.embed(inpt)
        feature, _ = self.lstm(embedded)
        oupt = self.linear(feature)
        return oupt

In [70]:
device= 'cpu'
hidden_size = 256
device= torch.device('cpu')

In [71]:
import json
import os
import random
from os import path
from string import ascii_uppercase, digits, punctuation
import colorama
import numpy
import regex

In [72]:
VOCAB= ascii_uppercase+digits+punctuation+" \t\n"

In [73]:
VOCAB

'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n'

In [76]:
temp_text=''
for i in range(len(etfo)):
    temp_text=temp_text+etfo[i]
etfo=temp_text

In [75]:
# conver lower text to uppper text in ETFO
etfo= etfo.upper()

In [58]:
def get_test_data():
    
    text = etfo
    text_tensor = torch.zeros(len(text), 1, dtype=torch.long)
    text_tensor[:, 0] = torch.LongTensor([VOCAB.find(c) for c in text])

    return text_tensor.to(device)

text_tensor = get_test_data()

In [59]:
def pred_to_dict(text, pred, prob):
    res = {"company": ("", 0), "date": ("", 0), "address": ("", 0), "total": ("", 0)}
    keys = list(res.keys())

    seps = [0] + (numpy.nonzero(numpy.diff(pred))[0] + 1).tolist() + [len(pred)]
    for i in range(len(seps) - 1):
        pred_class = pred[seps[i]] - 1
        if pred_class == -1:
            continue

        new_key = keys[pred_class]
        new_prob = prob[seps[i] : seps[i + 1]].max()
        if new_prob > res[new_key][1]:
            res[new_key] = (text[seps[i] : seps[i + 1]], new_prob)

    return {k: regex.sub(r"[\t\n]", " ", v[0].strip()) for k, v in res.items()}

In [60]:
print(text_tensor.shape)
for i in range(len(text_tensor)-1):
  if text_tensor[i]<0 or text_tensor[i]>70:
    text_tensor = torch.cat([text_tensor[0:i], text_tensor[i+1:]])

torch.Size([340, 1])


In [61]:

def test():
    
    model = MyModel0(len(VOCAB), 16, hidden_size).to(device)
    # dataset = MyDataset(None, args.device, test_path="/content/gdrive/My Drive/ResearchPaper/KIPE/data/test_dict.pth")

    model.load_state_dict(torch.load("/content/model.pth"))

    model.eval()
  
    with torch.no_grad():
            oupt = model(text_tensor)
            prob = torch.nn.functional.softmax(oupt, dim=2)
            prob, pred = torch.max(prob, dim=2)
            prob = prob.squeeze().cpu().numpy()
            pred = pred.squeeze().cpu().numpy()
            real_text = etfo
            result = pred_to_dict(real_text, pred, prob)

            with open("/content/" + 'result' + ".json", "w", encoding="utf-8") as json_opened:
                json.dump(result, json_opened, indent=4)

            print(result)
            #print(key)


if __name__ == "__main__":
    test()

{'company': 'BOOK TA_K (TAMAN DAYA) SDN BHD', 'date': '25/12/2018', 'address': '57 & 58, JALAN SAGU 18,  TAMAN DAYA  81100 JOHOR BAHRU.  DB STEHT', 'total': ''}


In [63]:
model = MyModel0(len(VOCAB), 16, hidden_size).to(device)
# dataset = MyDataset(None, args.device, test_path="/content/gdrive/My Drive/ResearchPaper/KIPE/data/test_dict.pth")

model.load_state_dict(torch.load("/content/model.pth"))

model.eval()

with torch.no_grad():
        oupt = model(text_tensor)
        prob = torch.nn.functional.softmax(oupt, dim=2)
        prob, pred = torch.max(prob, dim=2)
        prob = prob.squeeze().cpu().numpy()
        pred = pred.squeeze().cpu().numpy()
        real_text = etfo

In [64]:
pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [65]:
text = real_text
res = {"company": ("", 0), "date": ("", 0), "address": ("", 0), "total": ("", 0)}
keys = list(res.keys())

seps = [0] + (numpy.nonzero(numpy.diff(pred))[0] + 1).tolist() + [len(pred)]
for i in range(len(seps) - 1):
    pred_class = pred[seps[i]] - 1
    #print(pred_class)
    if pred_class == -1:
        continue

    new_key = keys[pred_class]
    new_prob = prob[seps[i] : seps[i + 1]].max()
    #print(new_key)
    if new_prob > res[new_key][1]:
        print(seps)
        res[new_key] = (text[seps[i] : seps[i + 1]], new_prob)

[0, 14, 15, 46, 50, 116, 117, 127, 243, 271, 276, 340]
[0, 14, 15, 46, 50, 116, 117, 127, 243, 271, 276, 340]
[0, 14, 15, 46, 50, 116, 117, 127, 243, 271, 276, 340]
[0, 14, 15, 46, 50, 116, 117, 127, 243, 271, 276, 340]


In [66]:
text

'TAN WOON YANN \nBOOK TA_K (TAMAN DAYA) SDN BHD \n$5.57 & 58, JALAN SAGU 18, \nTAMAN DAYA \n81100 JOHOR BAHRU. \nDB STEHT \n25/12/2018 8:13:39 PM \nCASH BILL \nCODE/DESC \nQRY \nROUR DING ADJUSTMENT \nROUND: :D TOTAL (RM); \nCASH 9 \' 40.00 CHANGE 00: DO 0 GOODS SOLD ARE NOT RETURINAT AK \n"WAP, 00, \n?-00 \nHOPE, F, ROB ATE LS OO FA 7 0, GOAE YET. ATA! \n'

In [67]:
pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,